## Nested Cross Validation with Grid Search

In order to impliment feature selection, hyperparameter tuning, and model selection, one could split the data into a training set and a test set, and apply cross validation on the training set. However this method uses the same data to tune model parameters and evaluate model performance which yields an overly-optimistic score because information may “leak” into the model and overfit the data. We refer to http://scikit-learn.org/stable/auto_examples/model_selection/plot_nested_cross_validation_iris.html for an example. 

Hence if one where to impliment feature/hyperparameter/model selection prior to cross-validation, and the best combination of feature/hyperparameters for the best model is chosen to be the estimator, we end up with a biased estimator. To obtain an unbiased estimator, one could impliment feature selection, hyperparameter tuning, and model selection independently in each fold of the cross-validation. This is the concept of nested cross validation. Below is an excerpt from "Temporal Video Segmentation" by Christian Petersohn which provides pseudo-code for the algorithm. 

<img src="figures/nested_cross_validation.png">

If we just think about hyperparameter tuning for the moment, one could apply grid search (http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) and use the nested Scikit-Learn cross validation method by referring to http://scikit-learn.org/stable/tutorial/statistical_inference/model_selection.html. However there is a few drawbacks with doing it this way:

* When using this method, you cannot nest objects with parallel computing
* Even if I try to use parallel computing on the inner sklearn.model_selection.GridSearchCV method (that is $n_jobs \neq 1$, It creates a copy of the data for each job. One way to combat this is by using the pre_dispatch parameter, but still, on my personal system, it will not run.
* There is no implimentation of early stopping with the GridSearchCV method (for tools such as xgboost). I was able to find this hack (https://www.kaggle.com/c/liberty-mutual-group-property-inspection-prediction/forums/t/15235/xgboost-sklearn-api-gridsearch-early-stopping?forumMessageId=85447#post85447) but there were complaints that it didn't support stratified K-folds. Using stratified sampling is important for our problem because the target feature is imbalanced. 

For these reasons, I implemented my own nested cross validation with grid search, and the ability to use parallel computing, as well as being able to implement early stopping for xgboost. This can be seen below:

In [2]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
import itertools
from sklearn import model_selection,metrics 
from sklearn.metrics import accuracy_score
import xgboost
import time
import sklearn.datasets

X,Y = sklearn.datasets.make_classification(n_samples=1000)

# data is split in a stratified fashion into train and test sets
seed = 7
test_size = 0.30
X_train_final, X_test_final, y_train_final, y_test_final = model_selection.train_test_split(X, Y, test_size=test_size, random_state=seed)



outerK = 4
innerK = 2
early_stopping_rounds = 100
eval_metric = 'auc'

skf = StratifiedKFold(n_splits=outerK)
skf.get_n_splits(X_train_final, y_train_final)

parameters = {'nthread' : [-1],
              'objective':['binary:logistic'],
              'learning_rate': [0.1], #so called `eta` value
              'max_depth': [6,14],
              'min_child_weight': [1,7,13],
              'silent': [1],
              'n_estimators': [1000000], #number of trees, change it to 1000 for better results
              'seed': [1337]}



def nested_CV_GS_XGB(parameters):
    
    def my_product(dicts):
        product = [x for x in apply(itertools.product, dicts.values())]
        return [dict(zip(dicts.keys(), p)) for p in product]
        
    bestModels = []
    bestModelScores = []
    bestModelsOuterparams = []
    topModels = []
    outerCounter = 1
    for train_index, test_index in skf.split(X_train_final, y_train_final):
        X_train, X_test = X_train_final[train_index], X_train_final[test_index]
        y_train, y_test = y_train_final[train_index], y_train_final[test_index]
        
        skfInner = StratifiedKFold(n_splits=innerK)
        skf.get_n_splits(X_train, y_train)

        tempModels = []
        scores = []
        innerCounter = 1
        for params in my_product(parameters):
            model = xgboost.XGBClassifier(**params)
            for train_index_inner, test_index_inner in skf.split(X_train, y_train):
                print 'inside inner loop'
                X_train_inner, X_test_inner = X_train[train_index_inner], X_train[test_index_inner]
                y_train_inner, y_test_inner = y_train[train_index_inner], y_train[test_index_inner]
                model.fit(X_train_inner, y_train_inner,verbose = False, early_stopping_rounds=early_stopping_rounds, eval_metric=eval_metric, eval_set=[(X_test_inner, y_test_inner)])
                scores.append(model.best_score)   
            avgScore = float(sum(scores))/len(scores)
            tempModels.append([model.get_params(), avgScore])
            
            print 'Finished Inner Fold For Some Inner Model', innerCounter
            innerCounter+=1
        tempModels.sort(key=lambda x: int(x[1]))
        bestMod = tempModels[-1][0]
        bestModels.append(bestMod)
        

        
        outerModel = xgboost.XGBClassifier(**bestMod)
        outerModel.fit(X_train, y_train,verbose = False, early_stopping_rounds=early_stopping_rounds, eval_metric=eval_metric,
              eval_set=[(X_test, y_test)])

        bestModelsOuterparams.append([[bestMod], [outerModel.best_score]])
        bestModelScores.append(outerModel.best_score)
        topModels.append(outerModel)
        print 'Finished Outer Fold', outerCounter, 'Score:', outerModel.best_score
        outerCounter+=1
    avgBestModelScores = float(sum(bestModelScores))/len(bestModelScores)
    
    return avgBestModelScores,topModels,  bestModelsOuterparams


t0 = time.time()
roc_auc, topModels, bestModels = nested_CV_GS_XGB(parameters)
t1 = time.time()
print 'function took', float(t1-t0)/60, 'mins' 
print 'Estimated ROC_AUC:', roc_auc

           
for i in range(outerK):
    print i, 'model max_depth:', bestModels[i][0][0]['max_depth']
    print i, 'model min_child_weight:', bestModels[i][0][0]['min_child_weight']


inside inner loop
inside inner loop
inside inner loop
inside inner loop
Finished Inner Fold For Some Inner Model 1
inside inner loop
inside inner loop
inside inner loop
inside inner loop
Finished Inner Fold For Some Inner Model 2
inside inner loop
inside inner loop
inside inner loop
inside inner loop
Finished Inner Fold For Some Inner Model 3
inside inner loop
inside inner loop
inside inner loop
inside inner loop
Finished Inner Fold For Some Inner Model 4
inside inner loop
inside inner loop
inside inner loop
inside inner loop
Finished Inner Fold For Some Inner Model 5
inside inner loop
inside inner loop
inside inner loop
inside inner loop
Finished Inner Fold For Some Inner Model 6
Finished Outer Fold 1 Score: 0.980491
inside inner loop
inside inner loop
inside inner loop
inside inner loop
Finished Inner Fold For Some Inner Model 1
inside inner loop
inside inner loop
inside inner loop
inside inner loop
Finished Inner Fold For Some Inner Model 2
inside inner loop
inside inner loop
inside

For each fold, it looks like the parameters max_depth = 14 and min_child_weight = 13 produced the best results.